Title: Books on Tape - Author Classification from Dante and Shakespeare

Given a line from either Shakespeare or Dante, we will train a machine learning model that will accurately predict the author.

We will investigate three candidate models and compare their performance: Amazon’s BlazingText, Naive Bayes, and KNeighbors.

We have chosen two scenes from the Merchant of Venice by William Shakespeare and two scenes from the Divine Comedy by Dante Alighieri. These scenes will be transcribed using Amazon Transcribe. Word transcription data will then be processed and cleaned. Transcriptions with low confidence will be evaluated for removal. Data will then be explored and formatted.

Formatted data will be used to train models and features selected. A second set of data will be chosen for and processed for validation. Each model’s performance will be evaluated and compared.

Questions we are now considering/ Interesting thoughts:
1. How well can Amazon Transcribe transcribe the unique language of Shakespeare? Will transcriptions with low confidence turn out to be revealing language for classifying authors?
2. Shakespeare is written in iambic pentameter. Dante is originally written in Terza Rima and often translated into iambic pentameter. Is meter a revealing feature for author classification?  Can meter be used as a feature?
3. Features of interest - words common by one author or another; uniqueness as a feature; clusters of words; semantics as a feature - Dante will have a lot of “fiery, hell, burning”

In [3]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = 'crazycurlygirlbucket311' # Replace with your own bucket name if needed
print(bucket)
prefix = 'BookProphet' #Replace with the prefix under which you want to store the data if needed

arn:aws:iam::023375022819:role/service-role/AmazonSageMaker-ExecutionRole-20181029T121824
crazycurlygirlbucket311


In [7]:
# get the files
!wget https://etc.usf.edu/lit2go/audio/mp3/the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3
!wget https://etc.usf.edu/lit2go/audio/mp3/the-merchant-of-venice-014-merchant-of-venice-act-3-scene-1.600.mp3
!wget http://www.archive.org/download/divine_comedy_librivox/divinecomedy_longfellow_05_dante.mp3
!wget http://www.archive.org/download/divine_comedy_librivox/divinecomedy_longfellow_10_dante.mp3
     

--2019-03-13 16:05:47--  https://etc.usf.edu/lit2go/audio/mp3/the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3
Resolving etc.usf.edu (etc.usf.edu)... 131.247.120.45
Connecting to etc.usf.edu (etc.usf.edu)|131.247.120.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2744761 (2.6M) [audio/mpeg]
Saving to: ‘the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3.2’

the-merchant-of-ven 100%[===================>]   2.62M  6.48MB/s    in 0.4s    

2019-03-13 16:05:48 (6.48 MB/s) - ‘the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3.2’ saved [2744761/2744761]

--2019-03-13 16:05:48--  https://etc.usf.edu/lit2go/audio/mp3/the-merchant-of-venice-014-merchant-of-venice-act-3-scene-1.600.mp3
Resolving etc.usf.edu (etc.usf.edu)... 131.247.120.45
Connecting to etc.usf.edu (etc.usf.edu)|131.247.120.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6927279 (6.6M) [audio/mpeg]
Saving to: ‘the-mercha

In [8]:
# save to S3
MP3Location = prefix + '/MP3Files'

sess.upload_data(path='the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3', bucket=bucket, key_prefix=MP3Location)
sess.upload_data(path='the-merchant-of-venice-014-merchant-of-venice-act-3-scene-1.600.mp3', bucket=bucket, key_prefix=MP3Location)
sess.upload_data(path='divinecomedy_longfellow_05_dante.mp3', bucket=bucket, key_prefix=MP3Location)
sess.upload_data(path='divinecomedy_longfellow_10_dante.mp3', bucket=bucket, key_prefix=MP3Location)



's3://crazycurlygirlbucket311/BookProphet/MP3Files/divinecomedy_longfellow_10_dante.mp3'

In [14]:
#transcribe first Merchant chapter
from __future__ import print_function
import time
transcribe = boto3.client('transcribe')
job_name = "merchant1"
job_uri = "s3://crazycurlygirlbucket311/BookProphet/MP3Files/the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3"
output_bucket = "crazycurlygirlbucket311"
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat='mp3',
    LanguageCode='en-US',
    OutputBucketName=output_bucket
)
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
{'TranscriptionJob': {'TranscriptionJobName': 'merchant1again', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'en-US', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://crazycurlygirlbucket311/BookProphet/MP3Files/the-merchant-of-venice-005-merchant-of-venice-act-2-scene-1.589.mp3'}, 'Transcript': {'TranscriptFileUri': 'https://s3.amazonaws.com/crazycurlygirlbucket311/merchant1again.json'}, 'CreationTime': datetime.datetime(2019, 3, 13, 16, 29, 23, 133000, tzinfo=tzlocal()), 'CompletionTime': datetime.datetime(2019, 3, 13, 16

In [20]:
!aws transcribe list-transcription-jobs


{
    "TranscriptionJobSummaries": [
        {
            "TranscriptionJobName": "merchant1again",
            "CreationTime": 1552494563.133,
            "CompletionTime": 1552494685.745,
            "LanguageCode": "en-US",
            "TranscriptionJobStatus": "COMPLETED",
            "OutputLocationType": "CUSTOMER_BUCKET"
        }
    ]
}


In [19]:
!aws transcribe delete-transcription-job --transcription-job-name merchant1